In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/__init__.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_inference_server.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/templates.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/base_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/relay.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evalua

## Library

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor,LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor

In [3]:
train_path="/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"
test_path="/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv"

In [4]:
train_data=pd.read_csv(train_path)
test_data =pd.read_csv(test_path)

In [5]:
train_data 

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [6]:
train_data.describe()

,Id,Properties,Format,Time,Discrete,Realtime,Turns,Alternating,Simultaneous,Stochastic,...,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
count,233234.000000,233234.0,233234.0,233234.0,233234.0,233234.0,233234.0,233234.0,233234.0,233234.000000,...,233234.000000,233234.000000,233234.000000,233234.000000,233234.000000,2.332340e+05,233234.000000,233234.000000,233234.000000,233234.000000
mean,116616.500000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.075829,...,0.680055,0.650042,0.270158,0.018385,2863.386342,1.666635e+05,6.759735,2.333528,6.087586,0.044398
std,67329.000679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.264726,...,0.466456,0.476957,0.444042,0.134339,9558.007843,1.890782e+05,5.377815,4.625684,5.272134,0.622847
min,0.000000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.030000,5.690000e+00,0.000000,0.000000,0.000000,-1.000000
25%,58308.250000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,95.870000,4.047234e+04,1.000000,0.000000,1.000000,-0.333333
50%,116616.500000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,1.000000,1.000000,0.000000,0.000000,449.790000,1.054746e+05,6.000000,0.000000,6.000000,0.000000
75%,174924.750000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.000000,...,1.000000,1.000000,1.000000,0.000000,1275.520000,2.379821e+05,11.000000,2.000000,10.000000,0.533333
max,233233.000000,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,164877.370000,2.096745e+06,44.000000,30.000000,45.000000,1.000000


In [7]:
print(len(train_data["agent2"].unique()))

72


In [8]:
drop_column=[]
def drop_column_name(df):
    for column in df.columns:
        if df[column].value_counts().max()/233234 >= .95:
            drop_column.append(column)
    return drop_column
drop_column_name(train_data)
print (drop_column)

['Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating', 'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules', 'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire', 'TwoPlayer', 'Multiplayer', 'Cooperation', 'Team', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 'Container', 'Board', 'TriangleShape', 'DiamondShape', 'SpiralShape', 'CircleShape', 'PrismShape', 'StarShape', 'ParallelogramShape', 'SquarePyramidalShape', 'RectanglePyramidalShape', 'TargetShape', 'TriangleTiling', 'BrickTiling', 'SemiRegularTiling', 'CelticTiling', 'MorrisTiling', 'QuadHexTiling', 'CircleTiling', 'ConcentricTiling', 'SpiralTiling', 'AlquerqueTiling', 'MancalaStores', 'MancalaThreeRows', 'MancalaSixRows', 'MancalaCircular', 'AlquerqueBoardWithOneTriangle', 'AlquerqueBoardWithTwoTriangles', 'AlquerqueBoardWithFourTriangles', 'AlquerqueBoardWithEightTriangles', 'ThreeMensMorrisBoard', 'Thr

In [9]:
print (len(drop_column)) 

459


In [10]:
def summarize_dataframe_info(df):
    table=pd.DataFrame(columns=["Column","Data Type","Null count"])
    Dtype=""
    null=0
    for column in df.columns:
        Dtype=df[column].dtype
        null=df[column].isnull().sum()
        new_row = {"Column": column, "Data Type": Dtype,"Null count":null}
        table = pd.concat([table, pd.DataFrame([new_row])], ignore_index=True)
    return table
        
        